# SQL,Seaborn,HTML 


In [1]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

# Set seaborn color palette to 'viridis'
sns.set_palette('Blues')

# File path to the CSV file
file_path = 'https://raw.githubusercontent.com/guzmanwolfrank/Data-SQL/BankSQL/BankSQL/data/banking_data.csv'

# Read CSV file into pandas DataFrame
df = pd.read_csv(file_path)

# Connect to SQLite database (or create it)
conn = sqlite3.connect('banking_data.db')

# Convert DataFrame to SQL
df.to_sql('banking_data', conn, if_exists='replace', index=False)

# Function to run SQL query and return the result as a DataFrame
def run_query(query):
    return pd.read_sql_query(query, conn)

# Define SQL queries with corresponding questions
queries = [
    ("SELECT * FROM banking_data LIMIT 10;", "Select first 10 rows"),  # 1. Select first 10 rows
    ("SELECT COUNT(*) AS Total_Transactions FROM banking_data;", "Count the number of rows"),  # 2. Count the number of rows
    ("SELECT DISTINCT TransactionType FROM banking_data;", "Select distinct transaction types"),  # 3. Select distinct transaction types
    ("SELECT AVG(Amount) AS Avg_Amount FROM banking_data;", "Calculate the average transaction amount"),  # 4. Calculate the average transaction amount
    ("SELECT StateID, COUNT(*) AS Transaction_Count FROM banking_data GROUP BY StateID;", "Count number of transactions per state"),  # 5. Count number of transactions per state
    ("SELECT Currency, AVG(Amount) AS Avg_Amount FROM banking_data GROUP BY Currency;", "Average transaction amount per currency"),  # 6. Average transaction amount per currency
    ("SELECT TransactionDate, Amount FROM banking_data ORDER BY Amount DESC LIMIT 5;", "Top 5 transactions by amount"),  # 7. Top 5 transactions by amount
]

# Execute queries and store the results
results = [run_query(query[0]) for query in queries]

# Close the connection
conn.close()

# Create visualizations using seaborn
sns.set(style="whitegrid", palette="Blues")

# Save Seaborn images as JPEG files
output_folder = "output_images"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Generate and save Seaborn plots
for i, (query, question) in enumerate(queries, 1):
    plt.figure(figsize=(10, 6))
    if i == 6:
        plt.figure(figsize=(12, 6))
        sns.distplot(results[i-1]['Avg_Amount'], color='blue')
        plt.title("Distribution of Average Transaction Amount per Currency")
        plt.xlabel("Average Transaction Amount")
        plt.ylabel("Density")
        image_path = os.path.join(output_folder, "output.png")
    else:
        if len(results[i-1].columns) == 1:  # If there's only one column
            sns.barplot(data=results[i-1], x=results[i-1].columns[0], y=results[i-1].index, palette="Blues")
            plt.xlabel(results[i-1].columns[0])
        else:
            sns.barplot(data=results[i-1], x=results[i-1].columns[0], y=results[i-1].columns[1], palette="Blues")
            plt.xlabel(results[i-1].columns[0])
            plt.ylabel(results[i-1].columns[1])
        plt.title(question)
        image_path = os.path.join(output_folder, f"plot_{i}.jpg")
        if i == 5:  # Rotate x-axis labels for query 5 plot
            plt.xticks(rotation=90)
    plt.savefig(image_path)
    plt.close()

# Generate HTML content
html_content = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Banking Data Project</title>
    <style>
       body {
            font-family: Arial, sans-serif;
            padding: 20px;
        }
        a {
            color: grey;
            text-decoration: none;
        }

        a:hover {
            color: darkgrey;
        }

        a:visited {
            color: grey;
        }

        a:active {
            color: grey;
        }
        h1 {
            color: #333;
        }
        h2, h3, h4 {
            color: #444;
        }
        h5{
            font-size: 24px;
        }
        pre {
            background-color: #f4f4f4;
            padding: 10px;
            border: 1px solid #ddd;
        }
        .container {
            display: flex;
            flex-wrap: wrap;
            justify-content: space-between;
        }
        .column {
            width: 40%;
            margin-bottom: 80px;
            float: left;
        }
           .column2 {
            width: 40%;
            margin-bottom: 40px;
            float: right;
        }
               .column3 {
            width: ;
            margin-bottom: 40px;
          
        }
        .query {
            margin-bottom: 20px;
        }
        .query-title {
            font-size: 18px;
            font-weight: bold;
            color: #333;
            margin-bottom: 10px;
        }
        .query-result {
            border-collapse: collapse;
            width: 80%;
            font-size: 14px;
        }
        .query-result th, .query-result td {
            border: 1px solid #ddd;
            padding: 8px;
         
        }
        .query-result th {
            background-color: #f2f2f2;
        }
        .query-image {
            margin-top: 20px;
        }
        .query-image img {
            max-width: 100%;
        }
    </style>
</head>

<body>

    <div class="row">
        <div class="column3">
            <h1>Banking Data Project</h1>

            <h2>Overview</h2>

            <p>This project demonstrates the process of transforming a CSV file into a Looker Dashboard and SQL database.
                We can run queries on the data, and visualize these queries using Seaborn and Looker.</p>

            <p>Additionally, the project generates an HTML file with the Looker dashboard embedded, which can be placed
                in an AWS S3 bucket for easy access and sharing.</p>

            <h2>Features</h2>

            <ul>
                <li><strong>Data Transformation</strong>: Reads data from a CSV file and transforms it into a format
                    suitable for analysis.</li>
                <li><strong>SQL Integration</strong>: Loads the transformed data into an SQLite database and runs various
                    SQL queries.</li>
                <li><strong>Data Visualization</strong>: Uses Seaborn to create visual representations of the query
                    results.</li>
                <li><strong>Looker Dashboard</strong>: Creates a Looker dashboard for interactive data exploration.</li>
                <li><strong>HTML Generation</strong>: Generates an HTML file embedding the Looker dashboard.</li>
                <li><strong>Cloud Integration</strong>: The generated HTML file can be uploaded to an AWS S3 bucket for
                    online access.</li>
            </ul>

            <h2>Project Structure</h2>

            <ul>
                <li><code>data/</code>: Contains the original CSV file and any additional data files.</li>
                <li><code>Notebooks/</code>: Project Code. Includes Python script for data transformation, SQL
                    operations, and visualization and HTML code.</li>
                <li><code>output_images/</code>: Stores generated output images from Seaborn plots.</li>
                <li><code>README.md</code>: Project documentation.</li>
                <li><code>banking_data_analysis.html</code>: Project HTML with embedded Looker Dashboard.</li>
            </ul>

            <h3>Prerequisites</h3>

            <ul>
                <li>Python 3.x</li>
                <li>Pandas</li>
                <li>SQLite3</li>
                <li>Seaborn</li>
                <li>Matplotlib</li>
                <li>Looker SDK</li>
                <li>AWS CLI (for S3 integration)</li>
            </ul>

            <h2>Data Dictionary</h2>

            <p>The CSV file contains the following columns related to the Banking CSV File:</p>

            <ul>
                <li><strong>TransactionID</strong>: Unique identifier for each transaction</li>
                <li><strong>AccountID</strong>: Unique identifier for each account</li>
                <li><strong>TransactionDate</strong>: Date of the transaction</li>
                <li><strong>Amount</strong>: Amount of money moved in the transaction</li>
                <li><strong>TransactionType</strong>: Type of transaction (e.g., deposit, withdrawal)</li>
                <li><strong>Description</strong>: Description of the transaction</li>
                <li><strong>First Name</strong>: First name of the account holder</li>
                <li><strong>Last Name</strong>: Last name of the account holder</li>
                <li><strong>VendorID</strong>: Unique identifier for each vendor</li>
                <li><strong>FeeID</strong>: Unique identifier for each fee</li>
                <li><strong>FeePayable</strong>: Amount of fee payable</li>
                <li><strong>Card</strong>: Type of card used (e.g., Virtual, Physical)</li>
                <li><strong>MCC GroupName</strong>: Merchant Category Code group name</li>
                <li><strong>Channel</strong>: Channel through which the transaction was made</li>
                <li><strong>CardState</strong>: State of the card (e.g., active, inactive)</li>
                <li><strong>CardToken</strong>: Tokenized representation of the card</li>
            </ul>
        </div>

        <div class="column3">
        <iframe width="600" height="450" src="https://lookerstudio.google.com/embed/reporting/d1e85f0d-9a43-4aab-ba44-d898cfa25feb/page/ZfY0D" frameborder="0" style="border:0" allowfullscreen sandbox="allow-storage-access-by-user-activation allow-scripts allow-same-origin allow-popups allow-popups-to-escape-sandbox"></iframe>
            <h2>SQL Queries</h2>

            <p>queries = </p>
            <ul>
                <li>Select first 10 rows: <code>SELECT * FROM banking_data LIMIT 10;</code></li>
                <li>Count the number of rows: <code>SELECT COUNT(*) AS Total_Transactions FROM banking_data;</code></li>
                <li>Select distinct transaction types: <code>SELECT DISTINCT TransactionType FROM banking_data;</code>
                </li>
                <li>Calculate the average transaction amount: <code>SELECT AVG(Amount) AS Avg_Amount FROM banking_data;
                    </code></li>
                <li>Count number of transactions per state: <code>SELECT StateID, COUNT(*) AS Transaction_Count FROM
                        banking_data GROUP BY StateID;</code></li>
                <li>Average transaction amount per currency: <code>SELECT Currency, AVG(Amount) AS Avg_Amount FROM
                        banking_data GROUP BY Currency;</code></li>
                <li>Top 5 transactions by amount: <code>SELECT TransactionDate, Amount FROM banking_data ORDER BY
                        Amount DESC LIMIT 5;</code></li>
            </ul>


            <h2>Findings</h2>

            <p>We can see that Florida had the highest number overall of transactions while USD CASH had the highest
                average transaction amount per currency group.</p>

            <p>It is also worthy to note that Withdrawal was the most popular transaction type in the data.  Crypto also had the highest average transaction over Cash and Vendor currency.</p>

            <h2>Conclusion</h2>

            <p>In conclusion, our analysis of the banking and money movement data reveals some interesting insights.
                Florida emerged as the state with the highest number of transactions overall, indicating a significant
                volume of financial activity in the region. Additionally, USD CASH stood out with the highest average
                transaction amount per currency group, suggesting that transactions involving this currency tend to be
                larger on average.</p>

            <p>Furthermore, the dominance of Withdrawal as the most popular transaction type highlights a common
                financial behavior among account holders. This finding underscores the importance of understanding
                customer preferences and behaviors to tailor financial services effectively.</p>

            <p>Overall, this project demonstrates the value of data analysis in uncovering patterns and trends within
                financial datasets, providing valuable insights that can inform business strategies and decision-making
                processes.</p>

            <h2>License</h2>

            <p>MIT License</p>

            <p>Copyright (c) 2024 Wolfrank Guzman</p>

       
        </div>
    </div>
</body>

</html>

"""

# Append each query and its result to the HTML content
for i, (query, question) in enumerate(queries, 1):
    if i not in [2, 4]:
        html_content += f"""
        <div class="column3">
            <div class="query">
                <div class="query-title">Query {i}: {question}</div>
                <table class="query-result">
                    <tr>
                        <th>{results[i-1].columns[0]}</th>
                        {'<th>' + results[i-1].columns[1] + '</th>' if len(results[i-1].columns) > 1 else ''}
                    </tr>
        """
        for _, row in results[i-1].iterrows():
            html_content += f"""
                    <tr>
                        <td>{row[0]}</td>
                        {'<td>' + str(row[1]) + '</td>' if len(results[i-1].columns) > 1 else ''}
                    </tr>
            """
        html_content += """
                </table>
            </div>
        """
        # Generate the image path for the current query
        if i == 6:
            image_path = os.path.join(output_folder, "output.png")
        else:
            image_path = os.path.join(output_folder, f"plot_{i}.jpg")
        
        # Embed Seaborn images into HTML
        html_content += f"""
            <div class="query-image">
                <img src="{image_path}" alt="Plot {i}">
            </div>
        </div>
        """

# Close the HTML content
html_content += """
</div>
</body>
</html>
"""

# Save HTML content to a file
html_file_path = "banking_data_analysis.html"
with open(html_file_path, "w") as html_file:
    html_file.write(html_content)

print(f"HTML file saved to: {os.path.abspath(html_file_path)}")


HTTPError: HTTP Error 404: Not Found

In [ ]:
# Send output file to AWS S3 Bucket, add a script that checks or runs list of files in bucket! 